## Размещение баннеров

Описание:
 - Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.
 - Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.
 - Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.
Часть открытых данных есть, например, на сайте archive.org:
https://archive.org/details/201309_foursquare_dataset_umn

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import collections
import matplotlib.pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


 - Скачаем архив fsq.zip по указанной выше ссылке.
 - Преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас
 - Возьмем только первые 100 тыс. строк.

In [2]:
data = pd.read_table('checkins.dat', sep='|', skipinitialspace = True, header=[0], skiprows=[1])
data.dropna(axis='rows', inplace=True)
data.columns = data.columns.str.strip()
X=data[:100000];
X_coords = pd.concat([X.latitude,X.longitude],axis=1)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 - Необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. 

In [27]:
clustering = MeanShift(bandwidth=0.1)
clustering.fit(X_coords)
labels = clustering.labels_.tolist()
cluster_centers = clustering.cluster_centers_

In [88]:
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
print("number of estimated clusters : %d" % n_clusters_)

number of estimated clusters : 3230


 - Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [85]:
c = collections.Counter(labels)
cmost = c.most_common()
cval = list(c.values())
cmostn = [i for i in cmost if i[1]>=15]

In [86]:
finded_clusters = np.array([cluster_centers[i[0]] for i in cmostn])
#finded_clusters = np.array(cluster_centers)

In [ ]:
with open('cluster_close.txt','w') as file_obj:
    for i in range(finded_clusters.shape[0]):
        file_obj.writelines(str(finded_clusters[i,0]) + ',' + str(finded_clusters[i,1]) + '\n')

- 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов и запишем их в offices
- из получившихся 20 центров надо выбрать тот, который наименее удален от ближайшего к нему офиса. Ответ — есть широта и долгота этого центра

In [87]:
offices =np.array([[33.751277, -118.188740],[25.867736, -80.324116], [51.503016, -0.075479],[52.378894, 4.885084],[39.366487, 117.036146],[-33.868457, 151.205134]]);
dist=[];
for i in range(finded_clusters.shape[0]):
    dist.append(np.min(np.linalg.norm(offices-finded_clusters[i,:],axis=1)))
np.min(dist),np.argmin(dist),finded_clusters[np.argmin(dist)]    

(0.007834758163107856, 392, array([-33.86063043, 151.20477593]))